# Лабораторная работа No 9
## Алгоритмы кластерного анализа данных

In [1]:
import pandas as pd

# pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('colleges.csv')
df = df[['okrug', 'X', 'Y']]
display(df)

,okrug,X,Y
0,Юго-Восточный административный округ,"37,850588","55,685852"
1,Южный,"37,657122","55,713902"
2,Центральный административный округ,"37,625878","55,73442"
3,Южный административный округ,"37,654086","55,726615"
4,Южный административный округ,"37,695225","55,688223"
...,...,...,...
280,Центральный административный округ,"37,68165","55,74362"
281,Центральный административный округ,"37,684059","55,736391"
282,Центральный административный округ,"37,684201","55,736755"
283,Восточный административный округ,"37,816595","55,789968"


In [3]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный',
 'Южный административный округ'}

In [4]:
df.loc[df.okrug == 'Южный', 'okrug'] = 'Южный административный округ'

In [5]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный административный округ'}